Эта тетрадка даёт лучший скор, хоть на мой взгляд для этого не нужно много делать, т.к. пользоваться предобученными моделями с `huggingface` крайне просто. Так же пробовал линейные классификаторы, байесовские классификаторы, самописный `LSTM` на предобученных эмбедингах `fasttext` от команды `DeepPavlov`, всё показывало качество значительно хуже чем предобученные трансформеры.  

Так же пробовал обучаться напрямую на дифференциируемый аналог F1 в качестве лосса, но похоже из-за того, что баланс классов 50/50, здесь такой лосс не подойдет, т.к. градиенты по лоссу для объектов нулевого класса почти нулевые, и из-за этого предсказать единицы правильно гораздо важнее с точки зрения лосса.

In [1]:
import random
import torch
import pandas as pd
import numpy as np

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

from datasets import load_metric
from datasets import load_dataset

TRAIN_DATA_PATH = '../input/fakenews/train.tsv' #путь к файлу трейна на диске
TEST_DATA_PATH = '../input/fakenews/test.tsv' #путь к файлу теста на диске
AUG_DATA_PATH = '../input/fakenews/augmented_train.csv' #путь к файлу с аугментациями
MODEL_NAME = 'DeepPavlov/rubert-base-cased-sentence' #название предобученной модели

Для максимальной воспроизводимости фиксируем сиды.

In [2]:
def seed_all(seed_value):
    random.seed(seed_value) 
    np.random.seed(seed_value)
    torch.manual_seed(seed_value) 
    if torch.cuda.is_available() :
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) 
        torch.backends.cudnn.deterministic = True 
        torch.backends.cudnn.benchmark = False
        
seed_all(21)

Для аугментации текста была использована библиотка `nlpaug`, метод - перевод текста на другой язык и обратно. Т.к. процесс занимает много времени, в архиве прикладываю сразу файл 'train_augmented.csv'. С кодом, выполняющим аугментацию, можно ознакомиться ниже (в аргументах класса `BackTranslationAug` указываем две модели с `huggingface`, которые будут выполнять перевод между языками).

Так же пробовал аугментации в виде замены слов на близкие по смыслу (контекстные эмбеддинги из моделей трансформеров) и случайную перестановку слов в предложении, оба метода приводили к ухудшению качества.

In [3]:
# !pip install git+https://github.com/makcedward/nlpaug.git
# import nlpaug
# import nlpaug.augmenter.word as naw

# text = 'Каждый охотник желает знать, где сидит фазан.'

# back_translation_aug = naw.BackTranslationAug(
#     from_model_name='Helsinki-NLP/opus-mt-ru-en', 
#     to_model_name='Helsinki-NLP/opus-mt-en-ru',
#     device='cuda'
# )

# back_translation_aug.augment(text)

Подгружаем данные (мерджим исходный трейн файл и файл с аугментациями, плюс переводим все в .csv т.к. `huggingface` не дружит с .tsv).

In [4]:
def get_csvs():
    news = pd.read_csv(TRAIN_DATA_PATH, sep='\t')
    news_test = pd.read_csv(TEST_DATA_PATH, sep='\t')
    news_augmented = pd.read_csv(AUG_DATA_PATH)
    
    news_train = pd.concat([news_augmented, news])
    news_train.to_csv('train.csv', index=False)
    news_test.to_csv('test.csv', index=False)

get_csvs()

Создаем датасеты с помощью библиотеки `datasets` от `huggingface`.

In [5]:
def get_datasets():
    train_dataset = load_dataset('csv', data_files='./train.csv')['train']
    test_dataset = load_dataset('csv', data_files='./test.csv')['train']
    
    train_dataset = train_dataset.rename_column("title", "text")
    train_dataset = train_dataset.rename_column("is_fake", "label")
    test_dataset = test_dataset.rename_column("title", "text")
    test_dataset = test_dataset.remove_columns('is_fake')
    
    return train_dataset, test_dataset

train_dataset, test_dataset = get_datasets()

Подгружаем предобученные модели и токенайзер, я остановился на предобученной модели `rubert-base-cased-sentence` от команды `DeepPavlov`. Используем класс `AutoModelForSequenceClassification` и указываем нужное количество классов.

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

Токенизируем наши тексты и удаляем колонку с ними (результаты токенизации будут записаны в другие колонки).  
Не объявляем здесь паддинг, т.к. в цикле обучении будет сделан паддинг по каждому батчу до длины наиболее длинного экземпляра в батче, а не целиком для всего датасета.

In [7]:
def tokenize_function(data):
    return tokenizer(data["text"])

tokenized_train_dataset = train_dataset.map(tokenize_function)
tokenized_test_dataset = test_dataset.map(tokenize_function)

tokenized_train_dataset = tokenized_train_dataset.remove_columns('text')
tokenized_test_dataset = tokenized_test_dataset.remove_columns('text')

Подгружаем метрику F1 для валидации и пишем функцию для ее вычисления по выходам модели.

In [8]:
metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Используем класс `TrainingArguments` в который записываем параметры для обучения, который потом передадим инстансу класса `Trainer`.

In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=7,              
    per_device_train_batch_size=4,   
    per_device_eval_batch_size=64,   
    learning_rate=1e-5,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    seed=21,
)

Объявляем `Trainer` и запускаем обучение.

In [10]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args, #аргументы
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_train_dataset, #для валидации датасета нет
    compute_metrics=compute_metrics #метрики
)

trained = trainer.train()

Функция для получения меток классов и сохранение в файл.

In [22]:
def get_test_labels():
    test_predictions = trainer.predict(tokenized_test_dataset)
    labels = np.argmax(test_predictions.predictions, axis=-1)    
    return labels

def save_test_predictions():
    news_test = pd.read_csv(TEST_DATA_PATH, sep='\t')
    news_test.is_fake = get_test_labels()
    news_test.to_csv('predictions.tsv', sep='\t', index=False)
    
save_test_predictions()